<a href="https://colab.research.google.com/github/unicamp-dl/IA025_2022S1/blob/main/ex05/Marcus_Vinicius_Borela_de_Castro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [46]:
nome = 'Marcus Vinícius Borela de CAstro'

print(f'Meu nome é {nome}')

Meu nome é Marcus Vinícius Borela de CAstro


Este exercicío consiste em treinar no MNIST um modelo de duas camadas, sendo a primeira uma camada convolucional e a segunda uma camada linear de classificação.

Não podemos usar as funções torch.nn.Conv{1,2,3}d

## Importação das bibliotecas

In [47]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import random
import torch
import torchvision
from torchvision.datasets import MNIST

## Fixando as seeds

In [48]:
def inicializa_seed(num_semente:int=123):
  """
  É recomendado reiniciar as seeds antes de inicializar o modelo, pois assim
  garantimos que os pesos vao ser sempre os mesmos.
  fontes de apoio: 
      http://nlp.seas.harvard.edu/2018/04/03/attention.html
      https://github.com/CyberZHG/torch-multi-head-attention/blob/master/torch_multi_head_attention/multi_head_attention.py#L15
  """
  random.seed(num_semente)
  np.random.seed(num_semente)
  torch.manual_seed(num_semente)
  #torch.cuda.manual_seed(num_semente)
  #Cuda algorithms
  #torch.backends.cudnn.deterministic = True 

In [49]:
inicializa_seed(123)

## Define pesos iniciais

In [50]:
in_channels = 1
out_channels = 2
kernel_size = 5
stride = 3

# Input image size
height_in = 28  
width_in = 28

# Image size after the first convolutional layer.
height_out = (height_in - kernel_size - 1) // stride + 1
width_out = (width_in - kernel_size - 1) // stride + 1

initial_conv_weight = torch.FloatTensor(in_channels, out_channels, kernel_size, kernel_size).uniform_(-0.01, 0.01)
initial_conv_bias = torch.FloatTensor(out_channels,).uniform_(-0.01, 0.01)

initial_classification_weight = torch.FloatTensor(10, out_channels * height_out * width_out).uniform_(-0.01, 0.01)
initial_classification_bias = torch.FloatTensor(10,).uniform_(-0.01, 0.01)

In [51]:
print(f" height_out {height_out}, width_out {width_out}")

 height_out 8, width_out 8


## Dataset e dataloader

### Definição do tamanho do minibatch

In [52]:
batch_size = 50

### Carregamento, criação dataset e do dataloader

In [53]:
dataset_dir = '../data/'

dataset_train_full = MNIST(dataset_dir, train=True, download=True,
                           transform=torchvision.transforms.ToTensor())
print(dataset_train_full.data.shape)
print(dataset_train_full.targets.shape)

torch.Size([60000, 28, 28])
torch.Size([60000])


### Usando apenas 1000 amostras do MNIST

Neste exercício utilizaremos 1000 amostras de treinamento.

In [54]:
indices = torch.randperm(len(dataset_train_full))[:1000]
dataset_train = torch.utils.data.Subset(dataset_train_full, indices)

## Define os pesos iniciais

In [55]:
loader_train = torch.utils.data.DataLoader(dataset_train, batch_size=batch_size, shuffle=False)

In [56]:
print('Número de minibatches de trenamento:', len(loader_train))


Número de minibatches de trenamento: 20


In [57]:
x_train, y_train = next(iter(loader_train))
print("\nDimensões dos dados de um minibatch:", x_train.size())
print("Valores mínimo e máximo dos pixels: ", torch.min(x_train), torch.max(x_train))
print("Tipo dos dados das imagens:         ", type(x_train))
print("Tipo das classes das imagens:       ", type(y_train))


Dimensões dos dados de um minibatch: torch.Size([50, 1, 28, 28])
Valores mínimo e máximo dos pixels:  tensor(0.) tensor(1.)
Tipo dos dados das imagens:          <class 'torch.Tensor'>
Tipo das classes das imagens:        <class 'torch.Tensor'>


In [58]:
x_train.shape, y_train.shape

(torch.Size([50, 1, 28, 28]), torch.Size([50]))

In [59]:
x_train[0, 0, 0, ] # 1a linha da 1a amostra

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.])

In [60]:
x_train[0, 0,] # 28 linhas da 1a amostra

tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000

In [61]:
x_train.shape[1] # canais

1

## Camada Convolucional

In [62]:
torch.zeros((4,1,4,5),  dtype=torch.float, requires_grad=True)

tensor([[[[0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.]]],


        [[[0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.]]],


        [[[0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.]]],


        [[[0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.]]]], requires_grad=True)

In [63]:
saida = torch.empty((4,1,4,5),  dtype=torch.float, requires_grad=True)

In [64]:
saida.shape

torch.Size([4, 1, 4, 5])

In [65]:
saida[0,0,0]

tensor([1.4613e-35, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
       grad_fn=<SelectBackward0>)

In [66]:
# torch.cat((saida[0,0,0], torch.tensor([[12]])), dim=-1)

In [67]:
# saida

In [177]:
class MyConv2d(torch.nn.Module):
  def __init__(self, in_channels: int, out_channels: int, kernel_size: int, stride: int):
    super(MyConv2d, self).__init__()

    self.in_channels = in_channels
    self.out_channels = out_channels
    self.kernel_size = kernel_size  # The same for height and width.
    self.stride = stride  # The same for height and width.
    self.weight = torch.nn.Parameter(torch.FloatTensor(in_channels, out_channels, kernel_size, kernel_size).uniform_(-0.01, 0.01))
    self.bias = torch.nn.Parameter(torch.FloatTensor(out_channels,).uniform_(-0.01, 0.01))
    print(f"Inicializado MyConv2d")
    print(f"in_channels: {self.in_channels} ")
    print(f"out_channels: {self.out_channels} ")
    print(f"kernel_size: {self.kernel_size} ")
    print(f"stride: {self.stride} ")
    print(f"weight.shape: {self.weight.shape} ")
    print(f"weight: {self.weight} ")
    print(f"bias.shape: {self.bias.shape} ")
    print(f"bias: {self.bias} ")

  def forward(self, x):
    assert x.dim() == 4, f'x must have 4 dimensions, not {x.shape}'
    assert x.shape[1] == 1, f'x must have only 1 channel, not {x.shape[1]}' # Num_canais sempre 1 (mnist, preto/branco)

    # print(f"kernel.shape: {self.weight.shape}, kernel: {self.weight}")
    # Escreva seu código aqui.
    # versão com for nas dimensões de X
    num_amostras = x.shape[0]
    num_linhas_entrada = x.shape[2]
    num_colunas_entrada = x.shape[3]
    num_linhas_saida = (num_linhas_entrada - self.kernel_size) // self.stride + 1
    num_colunas_saida = (num_colunas_entrada - self.kernel_size) // self.stride + 1
    print(f" num_amostras: {num_amostras}, self.out_channels: {self.out_channels}, num_linhas_entrada: {num_linhas_entrada}, num_colunas_entrada: {num_colunas_entrada}, num_linhas_saida: {num_linhas_saida}, num_colunas_saida: {num_colunas_saida}")
    saida = torch.zeros((num_amostras,self.out_channels,num_linhas_saida,num_colunas_saida), dtype=torch.float, requires_grad=False)        
    print(f"saida.shape: {saida.shape}")
    for ndx_amostra in range(num_amostras):
      print(f"\nndx_amostra: {ndx_amostra}")
      for ndx_in_channels in range(self.in_channels):
        print(f"\nndx_in_channels: {ndx_in_channels}")
        for ndx_linhas_saida in range(num_linhas_saida):
          for ndx_colunas_saida in range(num_colunas_saida):
            print(f"\nndx_linhas_saida, ndx_colunas_saida: {ndx_linhas_saida}, {ndx_colunas_saida}")
            print(f" alvo do kernel em x: x[{ndx_amostra},{ndx_in_channels},{ndx_linhas_saida}:{ndx_linhas_saida+self.kernel_size}, {ndx_colunas_saida}:{ndx_colunas_saida+self.kernel_size}]")
            print(f" \n {x[ndx_amostra, ndx_in_channels, ndx_linhas_saida:ndx_linhas_saida+self.kernel_size, ndx_colunas_saida:ndx_colunas_saida+self.kernel_size]}")
            produto = torch.mul(x[ndx_amostra, ndx_in_channels, ndx_linhas_saida:ndx_linhas_saida+self.kernel_size, ndx_colunas_saida:ndx_colunas_saida+self.kernel_size], self.weight)
            print(f" produto: {produto}")
            soma = torch.sum(produto, dim=(2,3), keepdim=True )
            print(f" soma: {soma}")
            valor_soma = soma.squeeze()
            print(f" valor_soma: {valor_soma}")

            # saida = torch.cat((saida, soma))
            if self.out_channels > 1:  # soma é um com dimensões, como em torch.tensor([[[[ 46.]], [[134.]]]])
              for ndx_out_channels in range(self.out_channels):
                saida[ndx_amostra, ndx_out_channels, ndx_linhas_saida, ndx_colunas_saida] += valor_soma[ndx_out_channels]
                print(f" somado na saída em [{ndx_amostra}, {ndx_out_channels}, {ndx_linhas_saida}, {ndx_colunas_saida}] = {saida[ndx_amostra, ndx_out_channels, ndx_linhas_saida, ndx_colunas_saida]}")
            else: # soma é um tensor escalar, como em tensor(34.)
                saida[ndx_amostra, 0, ndx_linhas_saida, ndx_colunas_saida] += valor_soma
                print(f" somado na saída em [{ndx_amostra}, 0, {ndx_linhas_saida}, {ndx_colunas_saida}] = {saida[ndx_amostra, 0, ndx_linhas_saida, ndx_colunas_saida]}")

            ndx_colunas_saida += self.stride
          ndx_linhas_saida += self.stride
    print(f" saida: {saida}")
    # somando bias
    for ndx_amostra in range(num_amostras):
      print(f"\nndx_amostra: {ndx_amostra}")
      for ndx_out_channels in range(self.out_channels):
        saida[ndx_amostra, ndx_out_channels] += self.bias[ndx_out_channels]
    print(f" saida apos somar bias: {saida}")
    # versão com for no kernel

    return saida

## Compare se sua implementação está igual à do pytorch usando um exemplo simples

In [178]:
in_channels_dummy = 1
out_channels_dummy = 1
kernel_size_dummy = 2
stride_dummy = 1
x = torch.arange(30).float().reshape(1, 1, 5, 6)

In [179]:
print(x)

tensor([[[[ 0.,  1.,  2.,  3.,  4.,  5.],
          [ 6.,  7.,  8.,  9., 10., 11.],
          [12., 13., 14., 15., 16., 17.],
          [18., 19., 20., 21., 22., 23.],
          [24., 25., 26., 27., 28., 29.]]]])


In [180]:

conv_layer = MyConv2d(in_channels=in_channels_dummy, out_channels=out_channels_dummy, kernel_size=kernel_size_dummy, stride=stride_dummy)

# Usa os mesmos pesos para minha implementação e a do pytorch
initial_weights_dummy = torch.arange(in_channels_dummy * out_channels_dummy * kernel_size_dummy * kernel_size_dummy).float()
initial_weights_dummy = initial_weights_dummy.reshape(in_channels_dummy, out_channels_dummy, kernel_size_dummy, kernel_size_dummy)
initial_bias_dummy = torch.arange(out_channels_dummy,).float()

conv_layer.weight.data = initial_weights_dummy
conv_layer.bias.data = initial_bias_dummy

out = conv_layer(x)


Inicializado MyConv2d
in_channels: 1 
out_channels: 1 
kernel_size: 2 
stride: 1 
weight.shape: torch.Size([1, 1, 2, 2]) 
weight: Parameter containing:
tensor([[[[0.0012, 0.0035],
          [0.0021, 0.0039]]]], requires_grad=True) 
bias.shape: torch.Size([1]) 
bias: Parameter containing:
tensor([-0.0098], requires_grad=True) 
 num_amostras: 1, self.out_channels: 1, num_linhas_entrada: 5, num_colunas_entrada: 6, num_linhas_saida: 4, num_colunas_saida: 5
saida.shape: torch.Size([1, 1, 4, 5])

ndx_amostra: 0

ndx_in_channels: 0

ndx_linhas_saida, ndx_colunas_saida: 0, 0
 alvo do kernel em x: x[0,0,0:2, 0:2]
 
 tensor([[0., 1.],
        [6., 7.]])
 produto: tensor([[[[ 0.,  1.],
          [12., 21.]]]], grad_fn=<MulBackward0>)
 soma: tensor([[[[34.]]]], grad_fn=<SumBackward1>)
 valor_soma: 34.0
 somado na saída em [0, 0, 0, 0] = 34.0

ndx_linhas_saida, ndx_colunas_saida: 0, 1
 alvo do kernel em x: x[0,0,0:2, 1:3]
 
 tensor([[1., 2.],
        [7., 8.]])
 produto: tensor([[[[ 0.,  2.],
     

In [181]:
pytorch_conv_layer = torch.nn.Conv2d(in_channels=in_channels_dummy, out_channels=out_channels_dummy, kernel_size=kernel_size_dummy, stride=stride_dummy, padding=0)
pytorch_conv_layer.load_state_dict(dict(weight=initial_weights_dummy, bias=initial_bias_dummy))
target_out = pytorch_conv_layer(x)

In [32]:
target_out.shape

torch.Size([1, 1, 4, 5])

In [182]:
assert torch.allclose(out, target_out, atol=1e-6)

## Compare se sua implementação está igual à do pytorch usando um exemplo aleatório

In [183]:
x = torch.rand(2, in_channels, height_in, width_in)
print(f"x.shape: {x.shape}, x:{x}")

x.shape: torch.Size([2, 1, 28, 28]), x:tensor([[[[0.0019, 0.0367, 0.7613,  ..., 0.2969, 0.0473, 0.2650],
          [0.9732, 0.9548, 0.3147,  ..., 0.9224, 0.9866, 0.6201],
          [0.3382, 0.2053, 0.6846,  ..., 0.2486, 0.9031, 0.8282],
          ...,
          [0.7676, 0.2166, 0.2992,  ..., 0.3895, 0.1751, 0.4259],
          [0.2837, 0.1579, 0.7146,  ..., 0.1176, 0.0715, 0.4455],
          [0.9980, 0.9825, 0.5886,  ..., 0.7503, 0.4703, 0.4557]]],


        [[[0.1217, 0.4824, 0.2845,  ..., 0.3949, 0.0027, 0.6096],
          [0.9385, 0.3673, 0.1423,  ..., 0.9886, 0.8045, 0.1761],
          [0.5097, 0.4730, 0.5628,  ..., 0.1195, 0.2930, 0.6708],
          ...,
          [0.0356, 0.0520, 0.2423,  ..., 0.3978, 0.4956, 0.2159],
          [0.0028, 0.4573, 0.1822,  ..., 0.5037, 0.4497, 0.7135],
          [0.3006, 0.8984, 0.7966,  ..., 0.9287, 0.6528, 0.0585]]]])


In [184]:
conv_layer = MyConv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, stride=stride)
conv_layer.weight.data = initial_conv_weight
conv_layer.bias.data = initial_conv_bias
out = conv_layer(x)


Inicializado MyConv2d
in_channels: 1 
out_channels: 2 
kernel_size: 5 
stride: 3 
weight.shape: torch.Size([1, 2, 5, 5]) 
weight: Parameter containing:
tensor([[[[ 0.0098, -0.0064, -0.0045, -0.0048,  0.0016],
          [ 0.0090,  0.0054, -0.0048, -0.0077,  0.0080],
          [ 0.0046, -0.0087,  0.0081, -0.0053,  0.0079],
          [ 0.0076, -0.0046,  0.0062, -0.0013, -0.0061],
          [-0.0079, -0.0043,  0.0016, -0.0034,  0.0097]],

         [[ 0.0042,  0.0026, -0.0078,  0.0061,  0.0053],
          [ 0.0081,  0.0038,  0.0020,  0.0093,  0.0006],
          [-0.0085, -0.0025, -0.0086,  0.0023,  0.0061],
          [ 0.0079, -0.0034,  0.0076, -0.0092, -0.0092],
          [ 0.0030,  0.0016,  0.0022, -0.0034, -0.0084]]]], requires_grad=True) 
bias.shape: torch.Size([2]) 
bias: Parameter containing:
tensor([0.0021, 0.0065], requires_grad=True) 
 num_amostras: 2, self.out_channels: 2, num_linhas_entrada: 28, num_colunas_entrada: 28, num_linhas_saida: 8, num_colunas_saida: 8
saida.shape: torch

In [191]:
out.shape

torch.Size([2, 2, 8, 8])

In [187]:
weight=initial_conv_weight.reshape(out_channels, in_channels, kernel_size, kernel_size )
weight.shape

torch.Size([2, 1, 5, 5])

In [188]:

# Usa os mesmos pesos para minha implementação e a do pytorch

pytorch_conv_layer = torch.nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, stride=stride, padding=0)
pytorch_conv_layer.load_state_dict(dict(weight=initial_conv_weight.reshape(out_channels, in_channels, kernel_size, kernel_size ), bias=initial_conv_bias))

target_out = pytorch_conv_layer(x)

In [190]:
target_out.shape

torch.Size([2, 2, 8, 8])

In [189]:
assert torch.allclose(out, target_out, atol=1e-6)

AssertionError: ignored

## Modelo

In [ ]:
class Net(torch.nn.Module):
    def __init__(self, height_in: int, width_in: int, in_channels: int, out_channels: int, kernel_size: int, stride: int):
        super(Net, self).__init__()
        self.conv_layer = MyConv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, stride=stride)
   
        height_out = (height_in - kernel_size - 1) // stride + 1
        width_out = (width_in - kernel_size - 1) // stride + 1
        self.classification_layer = torch.nn.Linear(out_channels * height_out * width_out, 10)

    def forward(self, x):
        hidden = self.conv_layer(x)
        hidden = torch.nn.functional.relu(hidden)
        hidden = hidden.reshape(x.shape[0], -1)
        logits = self.classification_layer(hidden)
        return logits

## Treinamento

### Definição dos hiperparâmetros

In [ ]:
n_epochs = 50
lr = 0.1

### Laço de treinamento

In [ ]:
model = Net(height_in=height_in, width_in=width_in, in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, stride=stride)

# Usa pesos iniciais pré-difinidos
model.classification_layer.load_state_dict(dict(weight=initial_classification_weight, bias=initial_classification_bias))
model.conv_layer.weight.data = initial_conv_weight
model.conv_layer.bias.data = initial_conv_bias

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr)

epochs = []
loss_history = []
loss_epoch_end = []
total_trained_samples = 0
for i in range(n_epochs):
    for x_train, y_train in loader_train:
        # predict da rede
        outputs = model(x_train)

        # calcula a perda
        loss = criterion(outputs, y_train)

        # zero, backpropagation, ajusta parâmetros pelo gradiente descendente
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_trained_samples += x_train.size(0)
        epochs.append(total_trained_samples / len(dataset_train))
        loss_history.append(loss.item())

    loss_epoch_end.append(loss.item())
    print(f'Epoch: {i:d}/{n_epochs - 1:d} Loss: {loss.item()}')


### Visualização usual da perda, somente no final de cada minibatch

In [ ]:
n_batches_train = len(loader_train)
plt.plot(epochs[::n_batches_train], loss_history[::n_batches_train])
plt.xlabel('época')

In [ ]:
loss_epoch_end

In [ ]:
# Assert do histórico de losses
target_loss_epoch_end = np.array([
    2.303267478942871,
    2.227701187133789,
    1.0923893451690674,
    0.5867354869842529,
    0.5144089460372925,
    0.45026642084121704,
    0.4075140357017517,
    0.37713879346847534,
    0.3534485101699829,
    0.3341451585292816,
    0.3181140422821045,
    0.30457887053489685,
    0.29283496737480164,
    0.2827608287334442,
    0.2738332152366638,
    0.2657742500305176,
    0.2583288848400116,
    0.25117507576942444,
    0.24439716339111328,
    0.23789969086647034,
    0.23167723417282104,
    0.22562651336193085,
    0.21984536945819855,
    0.2142913043498993,
    0.20894232392311096,
    0.203872948884964,
    0.19903430342674255,
    0.19439971446990967,
    0.18994088470935822,
    0.18563991785049438,
    0.18147490918636322,
    0.17744913697242737,
    0.17347246408462524,
    0.16947467625141144,
    0.16547319293022156,
    0.16150487959384918,
    0.1574639081954956,
    0.1534043848514557,
    0.14926929771900177,
    0.1452063024044037,
    0.1412365883588791,
    0.13712672889232635,
    0.1331038922071457,
    0.1291467249393463,
    0.1251506358385086,
    0.12116757035255432,
    0.11731722950935364,
    0.11364627629518509,
    0.11001908034086227,
    0.10655981302261353])

assert np.allclose(np.array(loss_epoch_end), target_loss_epoch_end, atol=1e-6)

## Rascunho

In [ ]:
in_channels_dummy = 1
out_channels_dummy = 2
kernel_size_dummy = 2
stride_dummy = 1
num_amostras_dummy = 1
x = torch.arange(30).float().reshape(num_amostras_dummy, 1, 5, 6)

In [ ]:
print(x)

tensor([[[[ 0.,  1.,  2.,  3.,  4.,  5.],
          [ 6.,  7.,  8.,  9., 10., 11.],
          [12., 13., 14., 15., 16., 17.],
          [18., 19., 20., 21., 22., 23.],
          [24., 25., 26., 27., 28., 29.]]]])


In [ ]:
# falta stride e tratar out_channels > 1

In [ ]:
temp = torch.tensor([[[[34.]]]])

In [ ]:
temp.squeeze()

tensor(34.)

In [ ]:
temp.squeeze().shape

torch.Size([])

In [ ]:
temp.squeeze().reshape(1,)

tensor([34.])

In [ ]:
temp.view(1,-1)

tensor([[34.]])

In [ ]:
temp = torch.tensor([[[[ 46.]], [[134.]]]])

In [ ]:
temp.squeeze()

tensor([ 46., 134.])

In [ ]:
temp.squeeze().shape

torch.Size([2])

In [ ]:
temp.squeeze().reshape(2,)

tensor([ 46., 134.])

In [ ]:
temp.squeeze().shape


torch.Size([2])

In [ ]:

conv_layer = MyConv2d(in_channels=in_channels_dummy, out_channels=out_channels_dummy, kernel_size=kernel_size_dummy, stride=stride_dummy)

# Usa os mesmos pesos para minha implementação e a do pytorch
initial_weights_dummy = torch.arange(in_channels_dummy * out_channels_dummy * kernel_size_dummy * kernel_size_dummy).float()
initial_weights_dummy = initial_weights_dummy.reshape(in_channels_dummy, out_channels_dummy, kernel_size_dummy, kernel_size_dummy)
initial_bias_dummy = torch.arange(out_channels_dummy,).float()

print(f"initial_weights_dummy.shape: {initial_weights_dummy.shape}")
conv_layer.weight.data = initial_weights_dummy
conv_layer.bias.data = initial_bias_dummy
print(f"conv_layer.weight.data: {conv_layer.weight.data}")
print(f"conv_layer.bias.data: {conv_layer.bias.data}")

out = conv_layer(x)


Inicializado MyConv2d
in_channels: 1 
out_channels: 2 
kernel_size: 2 
stride: 1 
weight.shape: torch.Size([1, 2, 2, 2]) 
weight: Parameter containing:
tensor([[[[ 1.2432e-03, -4.2557e-05],
          [ 2.7263e-03,  6.7489e-03]],

         [[-5.8188e-03,  8.4502e-03],
          [-7.0384e-03,  8.2780e-03]]]], requires_grad=True) 
bias.shape: torch.Size([2]) 
bias: Parameter containing:
tensor([-0.0028, -0.0097], requires_grad=True) 
initial_weights_dummy.shape: torch.Size([1, 2, 2, 2])
conv_layer.weight.data: tensor([[[[0., 1.],
          [2., 3.]],

         [[4., 5.],
          [6., 7.]]]])
conv_layer.bias.data: tensor([0., 1.])
 num_amostras: 1, self.out_channels: 2, num_linhas_entrada: 5, num_colunas_entrada: 6, num_linhas_saida: 4, num_colunas_saida: 5
saida.shape: torch.Size([1, 2, 4, 5])

ndx_amostra: 0

ndx_in_channels: 0

ndx_linhas_saida, ndx_colunas_saida: 0, 0
 alvo do kernel em x: x[0,0,0:2, 0:2]
 
 tensor([[0., 1.],
        [6., 7.]])
 produto: tensor([[[[ 0.,  1.],
       